In [3]:
from mlagents_envs.environment import UnityEnvironment, ActionTuple
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel

import numpy as np 

if __name__ == '__main__':
    # 환경 정의 및 설정 
    engine_configuration_channel = EngineConfigurationChannel()
    env = UnityEnvironment(file_name='환경의 경로', 
                           worker_id=np.random.randint(65535),
                           side_channels=[engine_configuration_channel])
    env.reset()

    # behavior 이름 불러오기 및 timescale 설정
    behavior_name = list(env.behavior_specs)[0]
    engine_configuration_channel.set_configuration_parameters(time_scale=1)

    # 전체 진행을 위한 반복문 (10 에피소드 반복)
    for ep in range(10):
        # 환경 초기화 
        env.reset()

        # decision_steps와 terminal_steps 정의
        decision_steps, terminal_steps = env.get_steps(behavior_name)

        # 파라미터 초기화 
        done = False
        ep_rewards = 0

        # 에피소드 진행을 위한 while문 
        while not done:
            # 랜덤 행동 설정
            random_action = np.random.randn(len(decision_steps),3)

            action_tuple = ActionTuple()
            action_tuple.add_continuous(random_action)

            env.set_actions(behavior_name, action_tuple)

            # 행동 수행 
            env.step()

            # 행동 수행 후 에이전트의 정보 (상태, 보상, 종료 여부) 취득
            decision_steps, terminal_steps = env.get_steps(behavior_name)
            
            done = len(terminal_steps.agent_id)>0
            reward = terminal_steps.reward[0] if done else decision_steps.reward[0]

            if done:
                next_state = [terminal_steps.obs[i][0] for i in range(6)]
            else:
                next_state = [decision_steps.obs[i][0] for i in range(6)]

            # 매 스텝 보상을 에피소드에 대한 누적보상에 더해줌 
            ep_rewards += reward 
        
        # 누적 보상 출력
        print('total reward for ep {} is {}'.format(ep, ep_rewards))

    # 환경 종료 
    env.close()

UnityEnvironmentException: Couldn't launch the 환경의 경로 environment. Provided filename does not match any environments.